In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from pathlib import Path

from SJ_Haar_CNV import base_search
from tests import test_data

In [2]:
X = np.array([0,0,0,1,1,1,2,2,2])
X

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

In [3]:
first_break = base_search.choose_break(X,0,X.size)
first_break

np.int64(8)

In [10]:
base_search.choose_break(X,3,X.size)

np.int64(8)

In [5]:
# I'm trying to manually figure out what's going on
l = X.size
s = 0
e = l
matrix = base_search.haar_matrix(s,e,l)
scores = np.matmul(matrix,X)
best_options = np.argwhere(scores == np.nanmax(scores)).flatten() + s + 1
best_options[np.abs(best_options - l //2).argmin()]

np.int64(8)

In [6]:
matrix

array([[ 0.63245553, -0.15811388, -0.15811388, -0.15811388, -0.15811388,
        -0.15811388, -0.15811388, -0.15811388, -0.15811388],
       [ 0.48304589,  0.48304589, -0.20701967, -0.20701967, -0.20701967,
        -0.20701967, -0.20701967, -0.20701967, -0.20701967],
       [ 0.38729833,  0.38729833,  0.38729833, -0.25819889, -0.25819889,
        -0.25819889, -0.25819889, -0.25819889, -0.25819889],
       [ 0.31622777,  0.31622777,  0.31622777,  0.31622777, -0.31622777,
        -0.31622777, -0.31622777, -0.31622777, -0.31622777],
       [ 0.25819889,  0.25819889,  0.25819889,  0.25819889,  0.25819889,
        -0.38729833, -0.38729833, -0.38729833, -0.38729833],
       [ 0.20701967,  0.20701967,  0.20701967,  0.20701967,  0.20701967,
         0.20701967, -0.48304589, -0.48304589, -0.48304589],
       [ 0.15811388,  0.15811388,  0.15811388,  0.15811388,  0.15811388,
         0.15811388,  0.15811388, -0.63245553, -0.63245553],
       [ 0.10540926,  0.10540926,  0.10540926,  0.10540926,  0

In [7]:
matrix.shape

(8, 9)

In [8]:
scores

array([-1.42302495, -1.86317701, -2.32379001, -2.21359436, -2.19469056,
       -2.27721635, -1.73925271, -1.15950181])

In [9]:
best_options

array([8])